In [25]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import time
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
import random
from scipy.sparse import hstack
from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier


In [26]:
def trans_to_df(filename):
    df_all=pd.DataFrame(columns=['title','author','institution','year','abstract','label'])
    f=open(filename,encoding='utf-8')
    i=0
    for line in f:
        if line!='\n':
            lines=line.split('\t')
            lines[5]=re.sub('\n','',lines[5])
#             print(lines[2])
#             print(lines[5])
            df_all=df_all.append({'author':lines[0],'abstract':lines[1],'title':lines[2],'institution':lines[3],'year':lines[4],'label':lines[5]},ignore_index=True)
    return df_all

In [27]:
df_all_raw_1=trans_to_df('result2.txt')
# df_all_raw_2=trans_to_df('result2.txt')
# df_all_raw=pd.concat([df_all_raw_1,df_all_raw_2])

df_all=df_all_raw_1
df_all=df_all[df_all['abstract']!='An abstract is not available.']
df_all=df_all.drop_duplicates()
df_all=df_all.reset_index(drop=True)
df_all['abstract'] = df_all['abstract']+df_all['title']*5+df_all['author']*5+df_all['institution']*5
# print(df_all[df_all['label']=='CIKM'].count())
# print(df_all[df_all['label']=='CHI'].count())
# print(df_all[df_all['label']=='KDD'].count())
# print(df_all[df_all['label']=='SIGIR'].count())
# print(df_all[df_all['label']=='SIGCSE'].count())
# print(df_all[df_all['label']=='WWW'].count())
# print(df_all[df_all['label']=='SIGGRAPH'].count())

In [28]:
def even(df1,label):
    df_label=df1[df1['label']==label]
#     display(df_label)
#     df_label=df_label.sample(n=range1,replace = False,random_state=42)
    return df_label


def nlp_preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ""
        # replace every special char with space
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', total_text)
        # replace multiple spaces with single space
        total_text = re.sub('\s+', ' ', total_text)
        total_text = re.sub(r"http\S+", "", total_text)
        # converting all the chars into lower-case.
        total_text = total_text.lower()
        total_text = re.sub(r"n\'t", " not", total_text)
        total_text = re.sub(r"\'re", " are", total_text)
        total_text = re.sub(r"\'s", " is", total_text)
        total_text = re.sub(r"\'d", " would", total_text)
        total_text = re.sub(r"\'ll", " will", total_text)
        total_text = re.sub(r"\'t", " not", total_text)
        total_text = re.sub(r"\'ve", " have", total_text)
        total_text = re.sub(r"\'m", " am", total_text)
#         for sub in set1:
#                 total_text = re.sub(sub, " ", total_text)
        for word in total_text.split():
            # if the word is a not a stop word then retain that word from the data
            if not word in stop_words:
                string += word + " "

        df_all[column][index] = string


In [29]:
def dict1(df_even):

    i=0
    list_author=[]
    list_author1=[]
    list_author2=[]
    list_author3=[]
    list_author4=[]
    list_author5=[]
    list_author6=[]
    sum=0
    list_author=df_even['author']
    dict1={}
    for list_author1 in list_author:
        list_author1=list_author1.split(':')
        for list_author2 in list_author1:
            list_author3.append(list_author2)
    list_author4=list(set(list_author3))[1:]
    for list_unique_author in list_author4:
        for list_author in list_author3:
            if list_unique_author== list_author: i+=1
        dict1[list_unique_author]=i
        i=0
    df_even['number_author']=0
    df_even['new_author_count']=0
    for index, row in df_even.iterrows():
        list_author5=row['author'].split(':')
        
        for list_author6 in list_author5:
            
            if not(list_author6)=='':
                sum=dict1[list_author6]+sum
            else:list_author5.remove('')
        df_even['number_author'][index]=len(list_author5)
        df_even['new_author_count'][index]=sum
        sum=0
   



In [30]:
#text processing stage.
stop_words = set(stopwords.words('english'))
start_time = time.clock()
for index, row in df_all.iterrows():
    if type(row['abstract']) is str:
        nlp_preprocessing(row['abstract'], index, 'abstract')
    else:
        print("there is no text description for id:",index)
print('Time took for preprocessing the text :',time.clock() - start_time, "seconds")

C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


Time took for preprocessing the text : 5.491975099999763 seconds


C:\Users\dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  if __name__ == '__main__':


In [31]:
print('CIKM',df_all[df_all['label']=='CIKM']['label'].count())
print('CHI',df_all[df_all['label']=='CHI']['label'].count())
print('KDD',df_all[df_all['label']=='KDD']['label'].count())
print('SIGIR',df_all[df_all['label']=='SIGIR']['label'].count())
print('SIGCSE',df_all[df_all['label']=='SIGCSE']['label'].count())
print('WWW',df_all[df_all['label']=='WWW']['label'].count())
print('siggraph',df_all[df_all['label']=='SIGGRAPH']['label'].count())

CIKM 1346
CHI 1046
KDD 594
SIGIR 1125
SIGCSE 1240
WWW 927
siggraph 1189


In [32]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [33]:
i=0
list_datafrme=[]
labels=df_all['label'].unique()
for label in labels: 
    i=i+1
    df_even=even(df_all,label)
    dict1(df_even)
    #     display(df_even)
    list_datafrme.append(df_even)

df_new_all=pd.concat(list_datafrme)
df_new_all.reset_index(drop=True)
stop_words = set(stopwords.words('english'))
df_new_all['year'] = df_new_all['year'].astype(int)

In [34]:
df_new_all.to_csv('train2.csv',index=False)

,title,author,institution,year,abstract,label,number_author,new_author_count
0,Sampling Big Trajectory Data,Yanhua Li:Chi-Yin Chow:Ke Deng:Mingxuan Yuan:J...,"Worcester Polytechnic Institute, Worcester, MA...",2015,increasing prevalence sensors mobile devices l...,CIKM,8,18
2882,A Clustering-based Approach to Detect Probable...,Daniel Lemes Gribel:Maira Gatti de Bayser:Leon...,Pontifical Catholic University of Rio de Janei...,2015,numerous lawsuits progress already judged braz...,CIKM,3,3
2883,VizQ: A System for Scalable Processing of Visi...,Arif Arman:Mohammed Eunus Ali:Farhana Murtaza ...,Bangladesh University of Engineering and Techn...,2017,demonstration present vizq efficient scalable ...,CIKM,4,4
2884,Identification of Microblogs Prominent Users d...,Imen Bizid:Nibal Nayef:Patrice Boursier:Sami F...,"L3i, University of La Rochelle, La Rochelle, F...",2015,specific real world events users microblogging...,CIKM,5,6
2885,Entity and Aspect Extraction for Organizing Ne...,Radityo Eko Prasojo:Mouna Kacimi:Werner Nutt:,"Free University of Bozen-Bolzano, Bozen-Bolzan...",2015,news websites give users opportunity participa...,CIKM,3,5
...,...,...,...,...,...,...,...,...
7462,A graph-theoretic approach to webpage segmenta...,Deepayan Chakrabarti:Ravi Kumar:Kunal Punera:,"Yahoo! Research, Sunnyvale, CA, USA:Yahoo! Res...",2008,consider problem segmenting webpage visually s...,WWW,3,15
7463,Fast maximum clique algorithms for large graphs,Ryan A. Rossi:David F. Gleich:Assefaw H. Gebre...,"Purdue University, West Lafayette, IN, USA:Pur...",2014,propose fast parallel maximum clique algorithm...,WWW,4,7
7464,Automatic Hierarchical Table of Contents Gener...,Debabrata Mahapatra:Ragunathan Mariappan:Vaibh...,"National University of Singapore, Singapore, S...",2018,number freely available online educational vid...,WWW,3,3
7465,Extrapolation methods for accelerating PageRan...,Sepandar D. Kamvar:Taher H. Haveliwala:Christo...,"Stanford University, Stanford, CA:Stanford Uni...",2003,present novel algorithm fast computation pager...,WWW,4,4
